In [1]:
from dataset_loader.uci_mhealth import samples_table, sensor_readings_table, DATASET_NAME
from dataset_util import preprocess
from dataset_util.preprocess import DEFAULT_WINDOW_SIZE, DEFAULT_WINDOW_OVERLAP
from misc import mul_str_arr
from config import SQLITE_DATABASE_FILE
import sqlite3
import pandas.io.sql as psql
import pandas as pd

raw_table_query = ("""
SELECT
    activity_id, timestamp, subject_id,
""" +
", ".join(mul_str_arr(["chest_acc"], ["x","y","z"]) +
    ["ecg_1", "ecg_2"] +
    mul_str_arr(["left_ankle", "right_lower_arm"],
                ["acc", "gyro", "magn"],
                ["x", "y", "z"])) + """
FROM
    {0}, {1}
WHERE {0}.sample_id = {1}.sample_id;
""").format(samples_table, sensor_readings_table)

In [2]:
conn = sqlite3.connect(SQLITE_DATABASE_FILE)

In [3]:
df = pd.read_sql(raw_table_query, conn)

In [4]:
df = df.iloc[:1000,:]

In [5]:
import datetime

def change_type(row):
    row['timestamp'] = datetime.datetime.utcfromtimestamp(float(row['timestamp'])).strftime("%M:%S.%f")
    return row
df = df.apply(change_type,axis = 1)
df

,activity_id,timestamp,subject_id,chest_acc_x,chest_acc_y,chest_acc_z,ecg_1,ecg_2,left_ankle_acc_x,left_ankle_acc_y,...,left_ankle_magn_z,right_lower_arm_acc_x,right_lower_arm_acc_y,right_lower_arm_acc_z,right_lower_arm_gyro_x,right_lower_arm_gyro_y,right_lower_arm_gyro_z,right_lower_arm_magn_x,right_lower_arm_magn_y,right_lower_arm_magn_z
0,0.0,00:00.000000,1.0,-9.8184,0.009971,0.29563,0.004186,0.004186,2.1849,-9.6967,...,0.299630,-8.6499,-4.5781,0.187760,-0.44902,-1.0103,0.034483,-2.350000,-1.610200,-0.030899
1,0.0,00:00.020000,1.0,-9.8489,0.524040,0.37348,0.004186,0.016745,2.3876,-9.5080,...,0.582980,-8.6275,-4.3198,0.023595,-0.44902,-1.0103,0.034483,-2.163200,-0.882540,0.326570
2,0.0,00:00.040000,1.0,-9.6602,0.181850,0.43742,0.016745,0.037677,2.4086,-9.5674,...,0.438510,-8.5055,-4.2772,0.275720,-0.44902,-1.0103,0.034483,-1.617500,-0.165620,-0.030693
3,0.0,00:00.060000,1.0,-9.6507,0.214220,0.24033,0.079540,0.117220,2.1814,-9.4301,...,0.575710,-8.6279,-4.3163,0.367520,-0.45686,-1.0082,0.025862,-1.077100,0.006945,-0.382620
4,0.0,00:00.080000,1.0,-9.7030,0.303890,0.31156,0.221870,0.205130,2.4173,-9.3889,...,0.445860,-8.7008,-4.1459,0.407290,-0.45686,-1.0082,0.025862,-0.536840,0.175900,-1.095500
5,0.0,00:00.100000,1.0,-9.6511,0.232610,0.42094,0.154890,0.138150,2.2639,-9.4493,...,0.010705,-8.7247,-4.0449,0.506090,-0.45686,-1.0082,0.025862,0.186740,0.720440,-0.364480
6,0.0,00:00.120000,1.0,-9.6269,0.029439,0.56054,0.004186,0.016745,2.1740,-9.6574,...,0.294050,-9.0864,-4.1474,0.261380,-0.42745,-1.0164,0.019397,0.359070,0.000091,0.368190
7,0.0,00:00.140000,1.0,-9.6422,0.293670,0.38433,-0.016745,-0.008373,2.2023,-9.4397,...,0.144150,-9.0143,-4.0052,0.476820,-0.42745,-1.0164,0.019397,0.536910,-0.168680,1.817500
8,0.0,00:00.160000,1.0,-9.8484,-0.001789,0.43142,-0.150710,-0.046049,2.2037,-9.6283,...,0.153180,-9.0469,-4.0475,0.245540,-0.42745,-1.0164,0.019397,0.348400,-1.067000,2.544700
9,0.0,00:00.180000,1.0,-9.7321,0.243260,0.28505,-0.590270,-0.385140,2.2135,-9.6887,...,0.297640,-8.8318,-4.1090,0.096632,-0.42745,-1.0164,0.019397,-0.195490,-1.595200,3.622200


In [6]:
df = df.set_index('timestamp')
df.index = pd.to_datetime(df.index,format = '%M:%S.%f')

In [9]:
def resample_raw_data(required_freq,df):
    try:
        df = df.set_index('timestamp')
    except:
        print('Already time series')
    
    req_period = 1/required_freq
    df = df.resample('{}S'.format(req_period)).interpolate(method='spline', order=4 )
    return df

In [10]:
resample_raw_data(100,df)

Already time series


,activity_id,subject_id,chest_acc_x,chest_acc_y,chest_acc_z,ecg_1,ecg_2,left_ankle_acc_x,left_ankle_acc_y,left_ankle_acc_z,...,left_ankle_magn_z,right_lower_arm_acc_x,right_lower_arm_acc_y,right_lower_arm_acc_z,right_lower_arm_gyro_x,right_lower_arm_gyro_y,right_lower_arm_gyro_z,right_lower_arm_magn_x,right_lower_arm_magn_y,right_lower_arm_magn_z
timestamp,,,,,,,,,,,,,,,,,,,,,
1900-01-01 00:00:00.000,0.0,1.0,-9.818400,0.009971,0.295630,0.004186,0.004186,2.184900,-9.696700,0.630770,...,0.299630,-8.649900,-4.578100,0.187760,-0.449020,-1.010300,0.034483,-2.350000,-1.610200,-0.030899
1900-01-01 00:00:00.010,0.0,1.0,-9.756527,0.081157,0.427388,-0.065676,-0.013789,2.123217,-9.620726,0.448132,...,0.430763,-8.768469,-4.186037,0.459844,-0.423931,-1.023124,0.107279,-1.039250,-0.338815,-1.315394
1900-01-01 00:00:00.020,0.0,1.0,-9.848900,0.524040,0.373480,0.004186,0.016745,2.387600,-9.508000,0.683890,...,0.582980,-8.627500,-4.319800,0.023595,-0.449020,-1.010300,0.034483,-2.163200,-0.882540,0.326570
1900-01-01 00:00:00.030,0.0,1.0,-9.755990,0.083945,0.432414,-0.064276,-0.014801,2.117967,-9.621196,0.445151,...,0.418040,-8.766556,-4.184827,0.455678,-0.424397,-1.022896,0.105676,-0.984526,-0.536569,-0.282864
1900-01-01 00:00:00.040,0.0,1.0,-9.660200,0.181850,0.437420,0.016745,0.037677,2.408600,-9.567400,0.681130,...,0.438510,-8.505500,-4.277200,0.275720,-0.449020,-1.010300,0.034483,-1.617500,-0.165620,-0.030693
1900-01-01 00:00:00.050,0.0,1.0,-9.755457,0.086730,0.437399,-0.062894,-0.015813,2.112753,-9.621660,0.442202,...,0.405406,-8.764734,-4.183558,0.451508,-0.424867,-1.022665,0.104080,-0.930649,-0.683709,0.528873
1900-01-01 00:00:00.060,0.0,1.0,-9.650700,0.214220,0.240330,0.079540,0.117220,2.181400,-9.430100,0.550310,...,0.575710,-8.627900,-4.316300,0.367520,-0.456860,-1.008200,0.025862,-1.077100,0.006945,-0.382620
1900-01-01 00:00:00.070,0.0,1.0,-9.754929,0.089512,0.442344,-0.061530,-0.016823,2.107575,-9.622119,0.439285,...,0.392862,-8.763002,-4.182231,0.447336,-0.425342,-1.022433,0.102491,-0.877780,-0.785514,1.142984
1900-01-01 00:00:00.080,0.0,1.0,-9.703000,0.303890,0.311560,0.221870,0.205130,2.417300,-9.388900,0.710980,...,0.445860,-8.700800,-4.145900,0.407290,-0.456860,-1.008200,0.025862,-0.536840,0.175900,-1.095500


In [11]:
df[df.index.duplicated()]

,activity_id,subject_id,chest_acc_x,chest_acc_y,chest_acc_z,ecg_1,ecg_2,left_ankle_acc_x,left_ankle_acc_y,left_ankle_acc_z,...,left_ankle_magn_z,right_lower_arm_acc_x,right_lower_arm_acc_y,right_lower_arm_acc_z,right_lower_arm_gyro_x,right_lower_arm_gyro_y,right_lower_arm_gyro_z,right_lower_arm_magn_x,right_lower_arm_magn_y,right_lower_arm_magn_z
timestamp,,,,,,,,,,,,,,,,,,,,,


In [13]:
df.index[-1]

Timestamp('1900-01-01 00:00:19.980000')

In [15]:
df.index.max()

Timestamp('1900-01-01 00:00:19.980000')